# Running hofx application in JEDI

## prerequisite 
- `jedi-bundle` is already built in your `JEDI_BUILD`.
- Background files
- observation files
- Input geometry files


---

## Introducton:

An H(x) (or hofx) application computes the model values at observation time/location. Depending on the observation type, this task can be as simple as extracting model values at the observation location. If the observed variable does not exist in the model outputs, it can be calculated using "observation operators" in the JEDI Unified Forward Operator (UFO) code. In the example here, the observed variable is Aerosol Optical Depth (AOD), which is not usually available in the models. The AOD observation operator will calculate AOD from the model state (using CRTM) to provide it to JEDI. 

Here, we discuss two approaches for computing H(x) in JEDI: Hofx3D and Hofx4D. In each approach, assumptions are made to reduce the computational cost. 

## HofX3D

<div style="text-align: center;">
    <img src="./figures/3DVar.jpg" width="500" height="300">
</div>

For the HofX3D approach, we take a model state that is valid at the middle of the assimilation window (blue circle). We also take all the observations measured within the assimilation window (red circles). Next, we must extract the model value at the observation time and location. Since computing the model state at every observation time is costly, we assume that all the observations (within the window) were measured at the same time as our model state, which is in the middle of the window. With this assumption, we only need to find the model value at the observation locations to compute the H(x) values. 

The JEDI executable for this application is `fv3jedi_hofx_nomodel.x`


### YAML structure (HofX3D):

```yaml
# Beginning and length of assimilation window
time window:
  begin: 2021-08-04T21:00:00Z
  length: PT6H

# Geometry of the model
geometry:
  fms initialization:
    namelist filename: inputs/geometry_input/fmsmpp.nml
  akbk: inputs/geometry_input/akbk72.nc4
  npx: 13
  npy: 13
  npz: 72
  field metadata override: inputs/geometry_input/geos.yaml

# Model state valid in the middle of the assimilation window
state:
  datetime: 2021-08-05T00:00:00Z
  filetype: cube sphere history
  datapath: inputs/geos_c12
  filenames: [aero_x48rt.abkg.eta.%yyyy%mm%dd_%hh%MMz.nc4, met_x48rt.bkg_clcv.%yyyy%mm%dd_%hh%MMz.nc4]
  state variables: [ud,vd,ua,va,t,delp,q,qi,ql,phis,SO4,
                    BCPHOBIC,BCPHILIC,BRPHOBIC,BRPHILIC,
                    DU,DU002,DU003,DU004,DU005,
                    SS,SS002,SS003,SS004]
# Observations measured throughout the assimilation window
# But are assumed to be valid in the middle of the assimilation window.
observations:
  observers:
  - obs space:
      name: Aod
      obsdatain:
        engine:
          type: H5File
          obsfile: inputs/obs/aod_viirs_obs_202108042100.nc4
      obsdataout:
        engine:
          type: H5File
          obsfile: output/hofx3d.aod_viirs_obs_202108042100.nc4
      simulated variables: [aerosolOpticalDepth]
      channels: 4
    obs operator:
      name: AodCRTM
      Absorbers: [H2O,O3]
      obs options:
        Sensor_ID: v.viirs-m_j1
        EndianType: little_endian
        CoefficientPath: crtm/
        AerosolOption: aerosols_gocart_default
        model units coeff: 1.0e9
    obs error:
      covariance model: diagonal
    get values:
      time interpolation: linear
```

A few points about this experiment's YAML file:
- Note that the assimilation window goes from `2021-08-04T21:00:00Z` to `2021-08-05T03:00:00Z`
- The model state is valid at `2021-08-05T00:00:00Z`, the middle of the window.
- If you examine the observation file (`aod_viirs_obs_202108042100.nc4`), you can see that it includes measurements from `21Z` to `3Z` throughout the assimilation window.
- The observation operator is `AodCRTM` which calculates AOD from the model state.
- The output is saved at `output/hofx3d.aod_viirs_obs_202108042100.nc4`. This file includes various variables including `hofx` and `obsValue`. 

### Running HofX3D
First, set the path to your `JEDI_BUILD` and `MPIEXEC`. Running `which mpiexec` will return the path to your `mpiexec`. `JEDI_BUILD` is the path to your JEDI build directory. 

Here is an example of setting these two variables:

```bash
export MPIEXEC=/discover/swdev/jcsda/spack-stack/openmpi-4.1.3/gcc-10.1.0/bin/mpiexec
export JEDI_BUILD=/discover/nobackup/mabdiosk/jedi-bundle/build-gnu-spack-1.5.1
```

All the files that you need are under `hofx` directory. So `cd hofx` and then you can run the application with this command: 

```bash
$MPIEXEC "-n" "6" $JEDI_BUILD/bin/fv3jedi_hofx_nomodel.x hofx_fv3-geos_aero_nomodel.yaml 2>&1 | tee log_hofx3d.txt
```

### examining the output of HofX3D run

examining the fb file and the log ....
maybe create some plots if infra is available

---

## HofX4D


<div style="text-align: center;">
    <img src="./figures/4DVar.jpg" width="500" height="300">
</div>


For the HofX4D approach, we take a model state that is valid at the beginning of the assimilation window (blue circle). We also take all the observations measured within the assimilation window (red circles). Next, we must extract the model value at the observation time and location. Computing the model state at every observation time using the full non-linear model is costly. Instead, we use a simplified (linear) model (dark blue arrows) to move the model state forward in time using a user-specified time step (`tstep`) and create smaller windows. Observations within each smaller window are assumed to be valid at the beginning of the window. Next, for each smaller window, model values at the observation locations are extracted to compute the H(x) values. 

The JEDI executable for this application is `fv3jedi_hofx.x`

### YAML structure (HofX4D):

```yaml
time window:
  begin: 2021-08-04T21:00:00Z
  length: PT6H
forecast length: PT6H
geometry:
  fms initialization:
    namelist filename: inputs/geometry_input/fmsmpp.nml
  akbk: inputs/geometry_input/akbk72.nc4
  npx: 13
  npy: 13
  npz: 72
  field metadata override: inputs/geometry_input/geos.yaml

model:
  name: FV3LM
  namelist filename: inputs/geometry_input/input_geos_c12.nml
  tstep: PT15M
  #tstep: PT6H # changing this impacts the result, does this tells us what the subwindows are?
  lm_do_dyn: 1
  lm_do_trb: 0
  lm_do_mst: 0
  model variables: [ua,va,t,delp,sphum,qi,ql,phis,SO4,
                    BCPHOBIC,BCPHILIC,BRPHOBIC,BRPHILIC,
                    DU,DU002,DU003,DU004,DU005,
                    SS,SS002,SS003,SS004]


initial condition:
  datetime: 2021-08-04T21:00:00Z
  filetype: cube sphere history
  datapath: inputs/geos_c12
  filenames: [aero_x48rt.abkg.eta.%yyyy%mm%dd_%hh%MMz.nc4, met_x48rt.bkg_clcv.%yyyy%mm%dd_%hh%MMz.nc4]
  state variables: [ud,vd,ua,va,t,delp,q,qi,ql,phis,SO4,
                    BCPHOBIC,BCPHILIC,BRPHOBIC,BRPHILIC,
                    DU,DU002,DU003,DU004,DU005,
                    SS,SS002,SS003,SS004]

observations:
  observers:
  - obs space:
      name: Aod
      obsdatain:
        engine:
          type: H5File
          obsfile: inputs/obs/aod_viirs_obs_202108042100.nc4
      obsdataout:
        engine:
          type: H5File
          obsfile: output/hofx4d.aod_viirs_obs_202108042100.nc4
      simulated variables: [aerosolOpticalDepth]
      channels: 4
    obs operator:
      name: AodCRTM
      Absorbers: [H2O,O3]
      obs options:
        Sensor_ID: v.viirs-m_j1
        EndianType: little_endian
        CoefficientPath: inputs/crtm/
        AerosolOption: aerosols_gocart_default
        model units coeff: 1.0e9
    obs error:
      covariance model: diagonal
    get values:
      time interpolation: linear

A few points about this experiment's YAML file:

<div class="alert alert-block alert-warning"> in progress </div>

### Running HofX4D

Note that for HofX4D we use `fv3jedi_hofx.x`

```bash
$MPIEXEC "-n" "6" $JEDI_BUILD/bin/fv3jedi_hofx.x hofx_fv3-geos_aero.yaml 2>&1 | tee log_hofx3d.txt
```

### examining the output of HofX4D run

<div class="alert alert-block alert-warning"> in progress </div>

---

## Comparing the two experiments

<div class="alert alert-block alert-warning"> in progress </div>

- compare run time
- compare the hofx values 

## Further reading and ctest examples

<div class="alert alert-block alert-warning"> in progress </div>

- You can use ctest `fv3jedi_test_tier1_hofx_fv3-geos_cf` in [fv3-jedi](https://github.com/JCSDA-internal/fv3-jedi/blob/1.8.0/test/CMakeLists.txt#L1107) as an [example](https://github.com/JCSDA-internal/fv3-jedi/blob/1.8.0/test/testinput/hofx_fv3-geos_cf.yaml). 

- Link to documentations and/or video tutorials